# Centroid Anomaly Detection
The following algorithm is based on Kloft M, Laskov P. <a href="http://proceedings.mlr.press/v9/kloft10a.html">Online Anomaly Detection under Adversarial Impact</a>. Proceedings of the Thirteenth International Conference on Artificial Intelligence and Statistics. 2010. pp. 405–412.

The centroid algorithm computes the mean of the set of "normal" examples. New points are judged as inliers or outliers based on their Euclidean distance from the mean. The appealing part of this algorithm is its simplicity, both in implementing the original training (finding the mean) and in updating the model. Updating can be done in two ways: one where the number of total data points keeps increasing, or one where the number of data points is held constant and previous data points are removed. The second way is called a finite sliding training window.

Kloft and Laskov found that using a finite sliding training window was resistant to poisoning attacks, so long as the attacker has limited access to the training data, which seems to be a reasonable assumption for real world attacks. This seems like an important consideration for learning algorithms, considering how <a href="http://www.evolvingai.org/fooling">image classification algorithms can be fooled</a>.

For the initial training, the only hyperparameter to tune is the threshold distance from the mean, $r$.
$$ class = 
\begin{cases}
    \text{inlier,} & d < r \\
    \text{outlier,} & d \geq r
\end{cases} $$
where $d$ is the Euclidean distance between the new point and the mean (centroid).

See ```Data Exploration and Setup.ipynb``` for more information on the data set used and how it was split.

## Initial Training

In [26]:
%matplotlib inline

In [27]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

In [28]:
#load training data
X_train = pd.read_csv('X_train.csv')

In [29]:
X_train.shape #should be (142403, 30)

(142403, 30)

Since I can't determine unique credit cards, I'm going to drop the 'Time' feature. I'm also going to mean normalize the data. This should make calculations a bit simpler.

In [30]:
X_train = X_train.drop('Time', 1)
X_train.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,-1.309615,0.118858,2.890224,3.259563,-0.975216,2.053227,-1.225558,1.224267,-0.036384,0.050075,...,0.301116,0.240538,0.707201,-0.251083,-0.450744,0.272565,0.567029,0.027397,-0.040843,82.24
1,-1.507604,-0.445468,0.383992,1.756901,-1.186740,0.489871,1.460411,0.438009,-0.487785,-0.816031,...,0.816683,0.340005,0.171648,0.960273,0.016460,-0.199284,-0.286972,-0.063034,0.074770,435.00
2,-3.579097,-3.780828,0.915284,0.811619,1.384262,-1.373492,-1.609020,0.818499,-1.445239,0.202760,...,0.866831,0.086957,-1.077871,0.391329,-0.448916,-0.352105,-0.870777,0.077323,-0.479606,189.00
3,-0.198195,-0.493945,1.132765,-0.409765,-0.872015,0.871469,0.279736,0.418671,0.962524,-0.824186,...,0.186004,-0.001488,-0.152296,0.564498,0.738859,-0.708118,0.285887,-0.040291,0.018017,187.10
4,-0.702706,1.432597,0.997674,0.906160,-0.158996,-0.563226,0.375988,0.414078,-0.757303,-0.289653,...,-0.018858,0.198920,0.667377,-0.051745,0.418866,-0.283271,-0.305879,0.312144,0.162622,2.18


In [31]:
X_train.shape

(142403, 29)

In [32]:
X_train = (X_train - X_train.mean()) / (X_train.std())

In [33]:
centroid = X_train.mean()

In [34]:
centroid.sum() #should be 0 or small

-7.983843702344881e-14

In [35]:
centroid.shape # should be (29,)

(29,)

Since this is an outlier detection algorithm, some good values of $r$ to try are multiples of $\sigma$, the standard deviation of the data points from the mean.

In [36]:
train_distances = np.linalg.norm(X_train, axis=1) # since centroid.sum() was so small, I'm going to round to zero

In [37]:
train_distances.shape # should be (142403,)

(142403,)

In [38]:
sigma = train_distances.std()

In [39]:
r_grid = np.arange(2, 6.1, 0.1) * sigma

In [40]:
# load validation data
X_val = pd.read_csv('X_val.csv')

In [41]:
X_val.shape #should be (56961, 30)

(56961, 30)

In [42]:
#drop 'Time' and mean normalize
X_val = X_val.drop('Time', 1)
X_val = (X_val - X_val.mean()) / (X_val.std())

In [43]:
X_val.shape # should be (56961, 29)

(56961, 29)

In [44]:
val_distances = np.linalg.norm(X_val, axis=1)

In [45]:
val_distances.shape # should be (56961,)

(56961,)

In [46]:
r = r_grid[0] #testing for for loop

In [47]:
val_distances[val_distances >= r].shape

(12344,)

In [48]:
y_val = pd.read_csv('y_val.csv', header=None)

In [49]:
y_val.shape #should be (56961, 1)

(56961, 1)

In [50]:
y_predicted = np.zeros_like(y_val)

In [51]:
y_predicted[np.where(val_distances >= r)] = 1

In [52]:
y_predicted.sum() #should be 12344

12344

I'll use the ```roc_auc_score()``` function from scikit learn to calculate the area under the curve.

In [53]:
auc = roc_auc_score(y_val, y_predicted)
auc

0.86995898773496894

In [54]:
auc = pd.DataFrame({'r': [], 'AUC': []})

for r in r_grid:
    row = {}
    row['r'] = r
    
    y_predicted = np.zeros_like(y_val)
    y_predicted[np.where(val_distances >= r)] = 1
    row['AUC'] = roc_auc_score(y_val, y_predicted)
    
    auc = auc.append(row, ignore_index=True)
auc

,AUC,r
0,0.869959,5.162541
1,0.878511,5.420668
2,0.889149,5.678795
3,0.899265,5.936922
4,0.906535,6.195049
5,0.914665,6.453176
6,0.917795,6.711303
7,0.919534,6.969430
8,0.922986,7.227557
9,0.923201,7.485684


In [55]:
auc.loc[auc['AUC'].idxmax()]

AUC    0.923201
r      7.485684
Name: 9, dtype: float64

92% just using Euclidean distance? Not bad! This algorithm is often improved by using a kernel, so I will also try that. The radial basis function kernel often does a good job:
$$ K(\mathbf{x}, \mathbf{x}') = exp(-\gamma||\mathbf{x} - \mathbf{x}'||^2) $$

In [56]:
# 1/n_features is a standard value for gamma, so I will try that first just for vizualization
gamma = 1 / X_train.shape[1]
train_dist_rbf = - gamma * train_distances.copy() ** 2
val_dist_rbf = - gamma * val_distances.copy() ** 2

In [57]:
sigma_rbf = train_dist_rbf.std()

In [58]:
r_grid = np.arange(2, 6.1, 0.1) * sigma

In [59]:
gamma_grid = np.arange(-20, 20) * 0.01 + gamma

In [60]:
auc_rbf = pd.DataFrame({'gamma': [], 'r': [], 'AUC': []})

for gamma in gamma_grid:
    val_dist_rbf = - gamma * val_distances.copy() ** 2
    train_dist_rbf = - gamma * train_distances.copy() ** 2
    sigma_rbf = train_dist_rbf.std()
    r_grid = np.arange(2, 6.1, 0.1) * sigma
    
    row = {}
    row['gamma'] = gamma
    
    for r in r_grid:
        row['r'] = r
        
        y_predicted = np.zeros_like(y_val)
        y_predicted[np.where(val_dist_rbf >= r)] = 1
        row['AUC'] = roc_auc_score(y_val, y_predicted)
        auc_rbf = auc_rbf.append(row, ignore_index=True)

In [61]:
auc_rbf['AUC'].describe()

count    1640.000000
mean        0.634919
std         0.172547
min         0.500000
25%         0.500000
50%         0.500000
75%         0.829318
max         0.924449
Name: AUC, dtype: float64

Only a 0.1% improvement. In that case, stick with simplicity. Now, let's see how it performs on the test data.

In [62]:
r = auc.loc[auc['AUC'].idxmax()].r
r

7.485684374791159

In [63]:
#load test data
X_test = pd.read_csv('X_test.csv')

In [64]:
X_test.shape #should be (56961, 30)

(56961, 30)

In [65]:
X_test = X_test.drop('Time', 1)

In [66]:
X_test.shape

(56961, 29)

In [67]:
X_test = (X_test - X_test.mean()) / (X_test.std())

In [68]:
test_distances = np.linalg.norm(X_test, axis=1)

In [69]:
test_distances.shape # should be (56961,)

(56961,)

In [70]:
y_test = pd.read_csv('y_test.csv', header=None)

In [71]:
y_test.shape #should be (56961, 1)

(56961, 1)

In [72]:
y_predicted = np.zeros_like(y_test)

In [73]:
y_predicted[np.where(test_distances >= r)] = 1

In [74]:
auc_final = roc_auc_score(y_test, y_predicted)
auc_final

0.90858154318377993

Looks like this model also generalizes well, which is encouraging.

## Updating the Model
The second task is updating the model as new data comes in. I will use the finite sliding training window for my algorithm based on the equations from Kloft and Laskov. For making updates, I will use the average-out method for choosing which points to remove. This removes the centroid and recalculates it as:
$$ \mathbf{c}' = \mathbf{c} + \frac {1} {n} (\mathbf{x} - \mathbf{c}) $$
where $\mathbf{c}$ is the old centroid vector, $\mathbf{c}'$ is the new centroid vector, $\mathbf{x}$ is the new data vector, and $n$ is the number of data vectors.

In [75]:
# to make the loops easier, I will make the original centroid vector into a vector of zeros
centroid[:] = 0
centroid.sum()

0.0

In [76]:
centroid.shape #should be (29,)

(29,)

In [77]:
# load update data
X_update = pd.read_csv('X_update.csv')

In [78]:
X_update.shape #should be (28482, 30)

(28482, 30)

In [79]:
X_update = X_update.drop('Time', 1)

In [80]:
X_update.shape

(28482, 29)

In [81]:
X_update = (X_update - X_update.mean()) / (X_update.std())

In [82]:
# first, see if majority of the points are predicted to be outliers
y_predicted = np.zeros(X_update.shape[0])

In [83]:
y_predicted.shape #should be (28482,)

(28482,)

In [84]:
update_distances = np.linalg.norm(X_update - centroid, axis=1)

In [85]:
y_predicted[np.where(update_distances >= r)] = 1

In [86]:
frac_outliers = y_predicted.sum() / y_predicted.shape[0]
frac_outliers

0.049399620813145147

In [87]:
if frac_outliers < 0.5: #it is in this example
    n = X_train.shape[0]
    
    for row in X_update.iterrows():
        centroid = centroid + (1 / n) * (row[1].values - centroid)

In [88]:
centroid.sum() #should not be exactly zero anymore

0.00010061674397263176

## Making Functions
I'm assuming that only the needed features are input into the function. I am not assuming that the data is normalized. For now, I'm only making sure these functions work with DataFrames, since that is what I used to develop it.

In [ ]:
# r was a multiple of sigma
multiplier = r / sigma
multiplier

In [1]:
def model_A_initialize(X_train):
    '''
    This function initializes the centroid anomaly detection algorithm. It makes sure that the initial data is mean
    normalized, which makes calculating the centroid trivial (it is zeros). It calculates the distance each data point
    is from the centroid and uses the standard devation of the distances to calculate r, the threshold distance for
    classifying points as outliers or inliers.
    
    Inputs:
        X_train: a DataFrame of features. All features will be used in the algorithm.
        
    Outputs:
        centroid: the centroid of the training data, an array of zeros
        r: the threshold distance from the centroid
        n: the number of training datapoints
    '''
    #imports
    import pandas as pd
    import numpy as np
    
    #mean normalize the input
    X_train = (X_train - X_train.mean()) / (X_train.std())

    # the centroid should be zero, so I'm hardcoding that to avoid rounding errors
    centroid = np.zeros(X_train.shape[1])
    
    # calculate r from data
    train_distances = np.linalg.norm(X_train, axis=1) #centroid is zero, so distance is just the norm
    multiplier = 2.9 #determined through validation
    sigma = train_distances.std() #the standard deviation of the distances
    r = multiplier * sigma #the distance threshold for outliers
    
    n = X_train.shape[0]
    
    return centroid, r, n

In [49]:
def model_A_update(X_update, centroid, r, n):
    '''
    This function updates the centroid model based on new data in X_update. First, it predicts which data points are
    outliers. It does this by calculating the distance of the points from the centroid. If the distance is greater than
    or equal to the threshold, r, then the point is an outlier. If a majority of the points are not outliers, the
    centroid gets updated.
    
    Inputs:
        X_update: a DataFrame of features. All features will be used in the algorithm.
        
    Outputs:
        centroid: the centroid of the training data
        r: the threshold distance from the centroid
        n: the number of training datapoints
        y_predicted: returns the predictions for the data point. 0=inlier, 1=outlier
    '''
    #imports
    import pandas as pd
    import numpy as np
    
    #make sure X_update is mean normalized
    X_update = (X_update - X_update.mean()) / (X_update.std())

    # first, see if majority of the points are predicted to be outliers
    update_distances = np.linalg.norm(X_update - centroid, axis=1)
    y_predicted = np.zeros(X_update.shape[0]) #start with zeros
    y_predicted[np.where(update_distances >= r)] = 1 #change to 1's where >= r
    frac_outliers = y_predicted.sum() / y_predicted.shape[0] #calculate fraction of outliers
    print(frac_outliers)

    if frac_outliers < 0.5: #if a majority of the points are not outliers
        #update the model
        for row in X_update.iterrows():
            centroid = centroid + (1 / n) * (row[1].values - centroid)
        print("centroid changed")
            
    return centroid, r, n, y_predicted

In [64]:
import pandas as pd
import numpy as np

In [65]:
#load training data
X_train = pd.read_csv('X_train.csv')

In [66]:
X_train = X_train.drop('Time', 1)

In [67]:
# load update data
X_update = pd.read_csv('X_update.csv')

In [68]:
X_update = X_update.drop('Time', 1)

In [69]:
centroid, r, n = model_A_initialize(X_train)

In [70]:
centroid.sum()

0.0

In [71]:
r # should be ~7.48568

7.4856843747911572

In [72]:
n #should be 142403

142403

In [73]:
centroid, r, n, y_predicted = model_A_update(X_update, centroid, r, n)

0.0493996208131
centroid changed


In [74]:
centroid.sum() #should not be zero

0.00010061674397263176

In [75]:
r #should not have changed

7.4856843747911572

In [76]:
n #should not have changed

142403

In [77]:
y_predicted.sum()

1407.0